# KoBERT finetuning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install torch
!pip install transformers

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [5]:
from tqdm import tqdm_notebook

In [6]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [22]:
from transformers import AutoTokenizer, AutoModel

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")
# bertmodel = AutoModel.from_pretrained("beomi/KcELECTRA-base")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [10]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/CUAI_Winter/train_.tsv", delimiter='\t')
valid_df = pd.read_csv("/content/drive/MyDrive/CUAI_Winter/valid_.tsv", delimiter='\t')

In [11]:
def tokens_to_sentence(tokens):
  return tokens.replace("'", '')[1:-1].split(', ')

In [12]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/CUAI_Winter/train_.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_valid = nlp.data.TSVDataset("/content/drive/MyDrive/CUAI_Winter/valid_.tsv", field_indices=[1,2], num_discard_samples=1)

In [13]:
print(dataset_train[0])

['아내 출산 되 신 나', '0']


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
# tok = get_tokenizer() # vocab path

In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        self.transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, vocab=vocab, max_seq_length=max_len, pad=pad, pair=pair)
        # self.tokenizer = bert_tokenizer
        self.sentences = [self.transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

    def clean(self, x):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        x = pattern.sub(' ', x)
        x = url_pattern.sub('', x)
        x = x.strip()
        x = repeat_normalize(x, num_repeats=2)
        return x
        
    def encode(self, x, **kwargs):
        return self.tokenizer.encode(
            str(x),
            padding='max_length',
            max_length=max_len,
            truncation=True,
            **kwargs,
        )


In [17]:
## Setting parameters
max_len = 230
batch_size = 10
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [24]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_valid = BERTDataset(dataset_valid, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [29]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [30]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [31]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
torch.cuda.empty_cache()

In [34]:
  import gc
  gc.collect()

33578

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/4088 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9448007345199585 train acc 0.2
epoch 1 batch id 201 loss 1.742891550064087 train acc 0.1696517412935325
epoch 1 batch id 401 loss 1.8331588506698608 train acc 0.1733167082294267
epoch 1 batch id 601 loss 1.8093633651733398 train acc 0.1703826955074874
epoch 1 batch id 801 loss 1.8607618808746338 train acc 0.1676654182272155
epoch 1 batch id 1001 loss 1.8215272426605225 train acc 0.16683316683316574
epoch 1 batch id 1201 loss 1.9112136363983154 train acc 0.16552872606161376
epoch 1 batch id 1401 loss 1.7885513305664062 train acc 0.16695217701641502
epoch 1 batch id 1601 loss 1.7555360794067383 train acc 0.16589631480324624
epoch 1 batch id 1801 loss 1.7868907451629639 train acc 0.16801776790671735
epoch 1 batch id 2001 loss 1.8051459789276123 train acc 0.16776611694152885
epoch 1 batch id 2201 loss 1.699618935585022 train acc 0.16901408450704225
epoch 1 batch id 2401 loss 1.769626259803772 train acc 0.16859641815910076
epoch 1 batch id 2601 loss 1.7126367092132

  0%|          | 0/513 [00:00<?, ?it/s]

epoch 1 test acc 0.17680311890838205


...

  0%|          | 0/513 [00:00<?, ?it/s]

epoch 9 test acc 0.17680311890838205


  0%|          | 0/4088 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 1.7743806838989258 train acc 0.3
epoch 10 batch id 201 loss 1.7844946384429932 train acc 0.18756218905472632
epoch 10 batch id 401 loss 1.7960546016693115 train acc 0.1793017456359104
epoch 10 batch id 601 loss 1.814205527305603 train acc 0.17670549084858544
epoch 10 batch id 801 loss 1.8082119226455688 train acc 0.1766541822721591
epoch 10 batch id 1001 loss 1.7718303203582764 train acc 0.17432567432567322
epoch 10 batch id 1201 loss 1.808786153793335 train acc 0.17144046627810014
epoch 10 batch id 1401 loss 1.7787113189697266 train acc 0.17337615988579433
epoch 10 batch id 1601 loss 1.808118224143982 train acc 0.17307932542161025
epoch 10 batch id 1801 loss 1.8054920434951782 train acc 0.17518045530260884
epoch 10 batch id 2001 loss 1.8087962865829468 train acc 0.17386306846576682
epoch 10 batch id 2201 loss 1.78256094455719 train acc 0.1734211721944573
epoch 10 batch id 2401 loss 1.7861416339874268 train acc 0.1730112453144529
epoch 10 batch id 2601 loss 1.7

  0%|          | 0/513 [00:00<?, ?it/s]